In [10]:
import pandas as pd
import sqlalchemy
from sqlalchemy.orm import sessionmaker
from pymongo import MongoClient
from datetime import datetime, timezone
from sqlalchemy import create_engine, Column, Float, String, Integer, DateTime
from sqlalchemy.orm import sessionmaker, declarative_base

In [11]:
Base = declarative_base()

class fx_data(Base):
    __tablename__ = 'fx_rates'
    id = Column(Integer, primary_key= True)
    pair = Column(String)
    fx_timestamp = Column(DateTime)
    rate = Column(Float)
    entry_timestamp = Column(DateTime, default=datetime.now(timezone.utc))

    def __init__(self, pair, rate, fx_timestamp, entry_timestamp):
        self.pair = pair
        self.rate = rate
        self.fx_timestamp = fx_timestamp
        self.entry_timestamp = entry_timestamp

In [13]:
engine = sqlalchemy.create_engine('sqlite:///fx_rates_sql.db')
Session = sqlalchemy.orm.sessionmaker(bind = engine)
session = Session()

data = session.query(fx_data).all()

df = pd.DataFrame(
    [(row.fx_timestamp, row.pair, row.rate, row.entry_timestamp) for row in data],
    columns=['fx_timestamp','pair','fx_rate','entry_timestamp']
)
df.to_csv('fx_rates_sql.csv',index=False)
session.close()

In [14]:
client = MongoClient('mongodb://localhost:27017/')
mongo_db = client['fx_rates_mg']
mongo_collection = mongo_db['fx']

pd.DataFrame(list(mongo_collection.find({}))).to_csv('fx_rates_mongo.csv',index=False)